In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
import string
from nltk.stem import WordNetLemmatizer
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import load_iris  # You can replace this with any classification dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


In [3]:
# Define the path to the downloaded NLTK data
nltk_data_path = '/path/to/your/nltk_data'

# Tell NLTK to look for data in the specified directory
nltk.data.path.append(nltk_data_path)

# Load the necessary resources
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()



In [4]:
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

In [5]:
train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [6]:
test

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [7]:
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

In [8]:
y = train[target_columns].idxmax(axis=1) 

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [9]:
y_encoded

array([0, 1, 2, ..., 0, 1, 0])

In [10]:
text=train["prompt"]+" "+train["response_a"]+" "+train["response_b"]

In [11]:
text

0        ["Is it morally right to try to have a certain...
1        ["What is the difference between marriage lice...
2        ["explain function calling. how would you call...
3        ["How can I create a test set for a very rare ...
4        ["What is the best way to travel from Tel-Aviv...
                               ...                        
57472    ["A simple mnemonic for \u03c0:\n\"How I wish ...
57473    ["In python, implement a naive Bayes with gaus...
57474    ["is it unethical to work on building weapons?...
57475    ["If a bait contains 0,0025% bromadiolon then ...
57476    ["three kids eat three apples in three days, h...
Length: 57477, dtype: object

In [12]:
test_data=test["prompt"]+" "+test["response_a"]+" "+test["response_b"]

## clean text 

In [13]:
def clean_text(text):
    # Replace escaped characters with their actual representations
    text = text.replace(r'\"', '"')
    text = text.replace(r"\'", "'")
    text = text.replace(r"\\n", " ")
    text = text.replace(r"\\/\\/", "//")
    
    # Remove code block markers
    text = re.sub(r'```.*?```', '', text, flags=re.DOTALL)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Function to remove stop words
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Function to apply stemming
def apply_stemming(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Combined preprocessing function
def preprocess_text(text):
    text = clean_text(text)
    text = remove_stop_words(text)
    text = apply_stemming(text)
    return text



In [14]:
clean_train=text.apply(preprocess_text)

In [15]:
clean_test=test_data.apply(preprocess_text)

## TFIDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
vectorizer = TfidfVectorizer()
vectorizer.fit(clean_train)

TfidfVectorizer()

In [18]:
X_train=vectorizer.transform(clean_train)

In [19]:
X_test=vectorizer.transform(clean_test)

In [20]:
random_state = 42


In [21]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_encoded, test_size=0.1, random_state=random_state)


## Model

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import xgboost as xgb

In [23]:
# print("Best parameters found: ", grid_search.best_params_)

# print("Best accuracy found: ", grid_search.best_score_)

## Best parameters found:  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 150}
##Best accuracy found:  0.3746715024752941

In [24]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', seed=random_state)

param_grid = {
    'n_estimators': 150,  # Should be an integer, not a list
    'max_depth': 8,       # Should be an integer, not a list
    'learning_rate': 0.01 # Should be a float, not a list
}



In [25]:
# Apply the parameters directly to the model
model.set_params(**param_grid)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_val)


In [26]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred, average='weighted')
recall = recall_score(y_val, y_pred, average='weighted')
f1 = f1_score(y_val, y_pred, average='weighted')
cm = confusion_matrix(y_val, y_pred)
report = classification_report(y_val, y_pred)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

Accuracy: 0.37
Precision: 0.38
Recall: 0.37
F1 Score: 0.37
Confusion Matrix:
 [[988 585 462]
 [971 552 401]
 [754 422 613]]
Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.49      0.42      2035
           1       0.35      0.29      0.32      1924
           2       0.42      0.34      0.38      1789

    accuracy                           0.37      5748
   macro avg       0.38      0.37      0.37      5748
weighted avg       0.38      0.37      0.37      5748



In [27]:
prediction = model.predict_proba(X_test)

In [28]:
submission = pd.DataFrame({
    'id': test['id'],  # Include the 'id' column from the test dataset
    'winner_model_a': prediction[:, 0],  # Probability predictions for winner_model_a
    'winner_model_b': prediction[:, 1],  # Probability predictions for winner_model_b
    'winner_tie': prediction[:, 2]  # Probability predictions for winner_tie
})

In [29]:
submission.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.321380,0.323238,0.355382
1,211333,0.337118,0.362554,0.300328
2,1233961,0.374254,0.334691,0.291055


In [30]:
submission.to_csv('submission.csv', index=False)